In [62]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from IPython.display import display

In [63]:
COMMODITY = 'copper'
METRIC = 'F1'

EDCR_RESULTS_PATH = f'{COMMODITY}/test/predictions/test_{METRIC}'
OG_RESULTS_PATH = f'{COMMODITY}/test/results_test.csv'

In [64]:
def extract_info_from_filename(filename):
    parts = filename.split("Rule")
    epsilon = parts[0].split()[-1]
    models_base = parts[1].split("for")
    models = models_base[0].strip()
    base = models_base[1].split("_predictions")[0].strip()
    return base, models, epsilon

In [65]:
def calculate_metrics(df):
    y_true = df['True']
    y_pred = df['Predicted']
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average=None, labels=[0, 1])
    prior = pd.read_csv(OG_RESULTS_PATH, nrows=1)['Prior'].iloc[0]
    return accuracy, precision, recall, f1, prior

In [66]:
def process_files(path):
    all_results = []  
    for filename in os.listdir(path):
        if filename.startswith('Confident') and filename.endswith("_predictions.csv"):
            base, models, epsilon = extract_info_from_filename(filename)
            df = pd.read_csv(os.path.join(path, filename))
            accuracy, precision, recall, f1, prior = calculate_metrics(df)
            row = {
                'Base': base,
                'Models': models,
                'Epsilon': epsilon,
                'Accuracy': accuracy,
                'Precision (0)': precision[0],
                'Recall (0)': recall[0],
                'F1 (0)': f1[0],
                'Precision (1)': precision[1],
                'Recall (1)': recall[1],
                'F1 (1)': f1[1],
                'Prior': prior  
            }
            all_results.append(row)
    
    # Convert the list of results to a DataFrame
    return pd.DataFrame(all_results)

In [67]:
results_df = process_files(EDCR_RESULTS_PATH)

c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

In [68]:
results_df_one = results_df[results_df['Models'] != 'all']
# pd.set_option('display.max_rows', 200)
results_df_one

,Base,Models,Epsilon,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior


### Base Model

In [69]:
results_df_all = results_df[(results_df['Models'] == 'all') & (results_df['Base'] != "dumb_spikes") & (results_df['Base'] != "dumb_non_spikes")]
results_df_all

,Base,Models,Epsilon,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior
0,CNNA_256_5,all,0.5,0.945946,0.978142,0.957219,0.967568,0.794872,0.885714,0.837838,0.16
3,CNNA_256_5,all,0.6,0.851351,0.870192,0.967914,0.916456,0.571429,0.228571,0.326531,0.16
6,CNNA_256_5,all,0.7,0.851351,0.856481,0.989305,0.918114,0.666667,0.114286,0.195122,0.16
9,CNNA_256_5,all,0.8,0.855856,0.857143,0.994652,0.920792,0.800000,0.114286,0.200000,0.16
12,CNNA_256_5,all,0.95,0.860360,0.857798,1.000000,0.923457,1.000000,0.114286,0.205128,0.16
15,CNNA_256_5,all,0.9,0.860360,0.857798,1.000000,0.923457,1.000000,0.114286,0.205128,0.16


### Dumb Model (Only Predicts Spikes)

In [70]:
results_df_all = results_df[(results_df['Models'] == 'all') & (results_df['Base'] == "dumb_spikes")]
results_df_all

,Base,Models,Epsilon,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior
2,dumb_spikes,all,0.5,0.229730,1.0,0.085561,0.157635,0.169903,1.0,0.290456,0.16
5,dumb_spikes,all,0.6,0.941441,1.0,0.930481,0.963989,0.729167,1.0,0.843373,0.16
8,dumb_spikes,all,0.7,0.990991,1.0,0.989305,0.994624,0.945946,1.0,0.972222,0.16
11,dumb_spikes,all,0.8,0.995495,1.0,0.994652,0.997319,0.972222,1.0,0.985915,0.16
14,dumb_spikes,all,0.95,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.000000,0.16
17,dumb_spikes,all,0.9,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.000000,0.16


### Dumb Model (Only Predicts Non-Spikes)

In [71]:
results_df_all = results_df[(results_df['Models'] == 'all') & (results_df['Base'] == "dumb_non_spikes")]
results_df_all

,Base,Models,Epsilon,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior
1,dumb_non_spikes,all,0.5,0.981982,0.979058,1.0,0.989418,1.0,0.885714,0.939394,0.16
4,dumb_non_spikes,all,0.6,0.869369,0.865741,1.0,0.928040,1.0,0.171429,0.292683,0.16
7,dumb_non_spikes,all,0.7,0.851351,0.850000,1.0,0.918919,1.0,0.057143,0.108108,0.16
10,dumb_non_spikes,all,0.8,0.842342,0.842342,1.0,0.914425,0.0,0.000000,0.000000,0.16
13,dumb_non_spikes,all,0.95,0.842342,0.842342,1.0,0.914425,0.0,0.000000,0.000000,0.16
16,dumb_non_spikes,all,0.9,0.842342,0.842342,1.0,0.914425,0.0,0.000000,0.000000,0.16


### Display All

In [72]:
og_df = pd.read_csv(OG_RESULTS_PATH)
og_df = og_df.drop('Unnamed: 0', axis=1)
pd.set_option('display.max_rows', 200)
og_df

,Name,Params,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),Prior
0,LSTM,256 layers,0.702703,0.853801,0.780749,0.815642,0.196078,0.285714,0.232558,0.16
1,LSTM,128 layers,0.229730,0.766667,0.122995,0.211982,0.145833,0.800000,0.246696,0.16
2,LSTM,64 layers,0.603604,0.827815,0.668449,0.739645,0.126761,0.257143,0.169811,0.16
3,LSTM,32 layers,0.689189,0.843023,0.775401,0.807799,0.160000,0.228571,0.188235,0.16
4,CNN with Attention,"32 filters, kernel size 7",0.608108,0.828947,0.673797,0.743363,0.128571,0.257143,0.171429,0.16
5,CNN with Attention,"32 filters, kernel size 5",0.585586,0.823129,0.647059,0.724551,0.120000,0.257143,0.163636,0.16
6,CNN with Attention,"32 filters, kernel size 3",0.626126,0.842105,0.684492,0.755162,0.157143,0.314286,0.209524,0.16
7,CNN with Attention,"64 filters, kernel size 7",0.774775,0.854922,0.882353,0.868421,0.241379,0.200000,0.218750,0.16
8,CNN with Attention,"64 filters, kernel size 5",0.621622,0.828025,0.695187,0.755814,0.123077,0.228571,0.160000,0.16
9,CNN with Attention,"64 filters, kernel size 3",0.387387,0.807229,0.358289,0.496296,0.136691,0.542857,0.218391,0.16
